### 作業1

---



練習將DHT22接於GPIO 27接腳，並且更改軟體的接腳設定值，重新執行範例程式，驗證在不同的接腳上安裝DHT22，程式一樣可以正確地讀出數值。


### **解答1**

---



In [ ]:
import Adafruit_DHT             # 匯入Adafruit_DHT模組
DHT_SENSOR = Adafruit_DHT.DHT22  # 感測器為DHT22
DHT_PIN = 27                    # 設定DHT22 Data接線的接腳位置
while True:
    humidity, temperature = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
    if humidity is not None and temperature is not None:
        print("Temp={0:0.1f}*C  Humidity={1:0.1f}%".format(temperature, humidity))
    else:
        print("Failed to retrieve data from humidity sensor")


將接腳改成17，記得需要把DHT22的資料線接到GPIO 17


In [ ]:
import Adafruit_DHT             # 匯入Adafruit_DHT模組
DHT_SENSOR = Adafruit_DHT.DHT22  # 感測器為DHT22
DHT_PIN = 17                    # 設定DHT22 Data接線的接腳位置
while True:
    humidity, temperature = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
    if humidity is not None and temperature is not None:
        print("Temp={0:0.1f}*C  Humidity={1:0.1f}%".format(temperature, humidity))
    else:
        print("Failed to retrieve data from humidity sensor")


### 作業2

---



觀察RPi.GPIO的程式碼，與GPIOZero程式碼對於繼電器控制上寫法的不同，如果我們要設定GPIO 26, 19, 13, 6四個接腳控制一個四路繼電器，練習實作一個GPIOZero四路繼電器的控制程式。


### **解答2**

---



要注意這段程式碼一定要在PI上面執行，要先安裝RPi.GPIO模組


In [ ]:
import RPi.GPIO as GPIO
from time import sleep

relay_pin = 26

GPIO.setmode(GPIO.BCM)
GPIO.setup(relay_pin, GPIO.OUT)
GPIO.output(relay_pin, 1)
try:
    while True:
        GPIO.output(relay_pin, 0)
        sleep(5)
        GPIO.output(relay_pin, 1)
        sleep(5)
except KeyboardInterrupt:
	pass
GPIO.cleanup()


GPIOZero的對應程式寫法

In [ ]:
import gpiozero

RELAY_PIN = 26
relay = gpiozero.OutputDevice(RELAY_PIN, active_high=False, initial_value=True)
try:
    while True:
      relay.off()
      sleep(5)
      relay.on()
      sleep(5)
except KeyboardInterrupt:
    print(“\n程式結束執行\n")
    # 跳出程式，程式執行結束
    sys.exit(0)

使用GPIO 26,19,13,6連接4個繼電器的程式碼

In [ ]:
import gpiozero

RELAY_PIN1 = 26
RELAY_PIN2 = 19
RELAY_PIN3 = 13
RELAY_PIN4 = 6
relay1 = gpiozero.OutputDevice(RELAY_PIN1, active_high=False, initial_value=True)
relay2 = gpiozero.OutputDevice(RELAY_PIN2, active_high=False, initial_value=True)
relay3 = gpiozero.OutputDevice(RELAY_PIN3, active_high=False, initial_value=True)
relay4 = gpiozero.OutputDevice(RELAY_PIN4, active_high=False, initial_value=True)
try:
    while True:
      relay1.off()
      relay2.off()
      relay3.off()
      relay4.off()
      sleep(5)
      relay1.on()
      relay2.on()
      relay3.on()
      relay4.on()
      sleep(5)
except KeyboardInterrupt:
    print(“\n程式結束執行\n")
    # 跳出程式，程式執行結束
    sys.exit(0)

### 作業3

---



將作業1與作業2結合，設定程式在溫度10度以下，打開GPIO26，溫度10度以上到20度之間，控制GPIO19，溫度20度到30度之間，控制GPIO13，溫度在30度以上，控制GPIO6，達成在不同的溫度區間時，控制不同的繼電器的需求。


In [ ]:
import gpiozero
import Adafruit_DHT             # 匯入Adafruit_DHT模組
RELAY_PIN1 = 26
RELAY_PIN2 = 19
RELAY_PIN3 = 13
RELAY_PIN4 = 6
relay1 = gpiozero.OutputDevice(RELAY_PIN1, active_high=True, initial_value=False)
relay2 = gpiozero.OutputDevice(RELAY_PIN2, active_high=True, initial_value=False)
relay3 = gpiozero.OutputDevice(RELAY_PIN3, active_high=True, initial_value=False)
relay4 = gpiozero.OutputDevice(RELAY_PIN4, active_high=True, initial_value=False)

DHT_SENSOR = Adafruit_DHT.DHT22 # 感測器為DHT22
DHT_PIN = 27          # 設定DHT22 Data接線的接腳位置
try:
    while True:
      humidity, temperature = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
      if humidity is not None and temperature is not None:
          print("Temp={0:0.1f}*C  Humidity={1:0.1f}%".format(temperature, humidity))
          if (temperature<=10):
            relay1.on()
            relay2.off()
            relay3.off()
            relay4.off()
          if (temperature>10 and temperature<=20):
            relay1.off()
            relay2.on()
            relay3.off()
            relay4.off()
          if (temperature>20 and temperature<=30):
            relay1.off()
            relay2.off()
            relay3.on()
            relay4.off()
          if (temperature>30):
            relay1.off()
            relay2.off()
            relay3.off()
            relay4.on()
      else:
          print("Failed to retrieve data from humidity sensor")
except KeyboardInterrupt:
    print(“\n程式結束執行\n")
    sys.exit(0)

GPIOZero範例程式碼，作為參考用

In [ ]:
import sys
import time

import gpiozero

RELAY_PIN = 26

# 開始時設定動作狀態為假(GPIO為0, active_high=False) 
# 初始狀態值設定為關閉: (initial_value=False)

relay = gpiozero.OutputDevice(RELAY_PIN, active_high=False, initial_value=False)

def set_relay(status):
    if status:
        print(“繼電器導通")
        relay.on()
    else:
        print(“繼電器斷路")
        relay.off()

def toggle_relay():
    print(“切換繼電器狀態, 開變關, 關變開")
    relay.toggle()

def main_loop():
    # 程式開始, 設定繼電器狀態為假 (GPIO輸出0)
    set_relay(False)
    while 1:
        # 切換繼電器狀態
        toggle_relay()
        # 等待一秒鐘時間 
        time.sleep(1)

if __name__ == "__main__":
    try:
        main_loop()
    except KeyboardInterrupt:
        # 如果按了鍵盤，執行底下的程式片段
        # 繼電器關閉
        set_relay(False)
        print(“\n程式結束執行\n")
        # 跳出程式，程式執行結束
        sys.exit(0)
